# Data prep

In [2]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format

Load data file from Canada open data portal and do  a preliminary cleanup

In [3]:
data_url = 'https://open.canada.ca/data/dataset/d8f85d91-7dec-4fd1-8055-483b77225d8b/resource/fac950c0-00d5-4ec1-a4d3-9cbebf98a305/download/contracts.csv'

# Set some data types and drop French columns
contracts = (pd.read_csv(data_url, 
                         parse_dates=['contract_date', 'delivery_date','contract_period_start'],
                         dtype = {'economic_object_code' : str, 
                                   'limited_tendering_reason':str,
                                   'award_criteria' : str,
                                   'article_6_exceptions':str,
                                   'trade_agreement_exceptions':str})
               .drop(columns=['description_fr','comments_fr', 'additional_comments_fr'])
            )

# Keep only English department name
contracts.owner_org_title = contracts.owner_org_title.str.split('|').str[0]

# Define categorical columns
cat_cols = ['economic_object_code','solicitation_procedure','indigenous_business',
           'indigenous_business_excluding_psib', 'land_claims', 'commodity_type', 'commodity_code','limited_tendering_reason',
            'instrument_type', 'award_criteria', 'socioeconomic_indicator', 'intellectual_property']
contracts[cat_cols] = contracts[cat_cols].astype('category')

contracts.agreement_type_code = contracts.agreement_type_code.str.strip().astype('category')

# Define boolean columns
bool_cols = ['potential_commercial_exploitation','former_public_servant', 'ministers_office']
contracts[bool_cols] = contracts[bool_cols].replace({'Y' : True, 'N' : False}).astype(bool)

# Clean initials from procurement ID, as these can mess up their uniqueness.
# Initials are sometimes added by procurement officer who approved it.
contracts.procurement_id = contracts.procurement_id.str.strip().str.replace('\/[A-Za-z]{2}$', '', regex=True)

# Get reporting period from reference number if NA
na_refs = contracts[contracts.reporting_period.isna()].reference_number.str.extract('(\d{4}\-\d{4}\-Q\d)')[0]
contracts.loc[na_refs.index, 'reporting_period'] = na_refs

# Standardize economic codes
contracts.economic_object_code = contracts.economic_object_code.str.zfill(4)

# Get fiscal year from contract date
contracts['fiscal_year'] = contracts.contract_date.dt.to_period('A-MAR').dt.year

C:\Users\Roberto\AppData\Local\Temp\ipykernel_9672\2542009004.py:3: DtypeWarning: Columns (3,4,14,15,16,17,18,19,20,21,22,23,24,27,28,29,30,31,32,33,34,35,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  contracts = (pd.read_csv(data_url,


This cell does a rough cleanup on the vendor names, removing legal suffixes. Adapted from the [R code by Sean Boots](https://github.com/GoC-Spending/contracts-data/blob/main/lib/vendors.R).

In [9]:
# Remove accented characters from vendor names, capitalize, remove punctuations

norms = (
 contracts.vendor_name
 .str.normalize('NFKD')
 .str.encode('ascii', errors='ignore')
 .str.decode('utf-8')
 .str.upper()
 .str.replace(r'[^\w\s]+', ' ', regex=True)
 .str.strip()
)

contracts['vendor_name_norm'] = norms

# Remove abbreviations form company names to better use the govcontracts normalization table
# Source: https://github.com/GoC-Spending/contracts-data/blob/main/lib/vendors.R#L26-L73

patterns = ["LIMITED",
    "LIMITEE",
    "LIMITE",
    "LIMITACE",
    "LTE",
    "LT",
    "LTEE",
    "LLP",
    "LP",
    "PLC",
    "LCC",
    "LLC",
    "INCORPORATED",
    "INC",
    "LTD",
    "LDT",
    "CO",
    "CORP",
    "CORPORATION",
    "INCORPORATION",
    "INCORP",
    "PLC",
    "PTY",
    "ULC",
    "LP",
    "AB",
    "SENC",
    "SENCRL",
    "SENCRLSRL",
    "SRL",
    "LLPSEN",
    "LTACE",
    "GMBH",
    "SA",
    "SPZOO",
    "SP ZOO",
    "SP Z OO",
    "SP Z O O",
    "BV",
    "B V",
    "SAS",
    "S A",
    "AG",
    "SCC",
    "S C C"]

# Add trailing and leading space to suffixes
patterns = [' ' + s + ' ' for s in patterns]

# Add a leading space to vendor names to prevent accidentally removing the pattern from the name itself
contracts.vendor_name_norm = contracts.vendor_name_norm + ' '

# make regex pattern for replacement 
repl_regex = r'(' + '|'.join(patterns) + ')'

contracts.vendor_name_norm = contracts.vendor_name_norm.str.replace(repl_regex, ' ', regex=True).str.replace('\s\s+', ' ', regex=True)

# Do a second pass in case the pattern remains
contracts.vendor_name_norm = contracts.vendor_name_norm.str.replace(repl_regex, '', regex=True).str.strip()

# Create unique procurement ID with buyer org name and first two words of vendor name
# in case of duplicates across departments
contracts['proc_unique'] = (contracts.procurement_id.str.lower() + '_' 
                            + contracts.owner_org.astype(str) + '_' 
                            + contracts.vendor_name_norm.str.split().str[0:2].str.join('_').str.lower()
                           )


In [ ]:
# Number of unique contracts since 2017

contracts.query('fiscal_year >= 2017').proc_unique.nunique()

475259

In [11]:
# Save prepped data to parquet for quicker loading

contracts.to_parquet('../data/contracts_prepped.parquet')

## Isolate amended contracts

This section isolates contracts that were amended. These tend to have the same procurement ID, vendor and purchasing department, but not always. A few methods were used to grassociate amended contracts with their original ones, again following the [methodology](https://govcanadacontracts.ca/methodology/#associate-amendments-with-original-contracts
) of the Carelton SPPA team.

In [10]:
# Isolate contracts that have more than one occurence of procurement ID, vendor and dept.
# Then do a second pass on the ungrouped records looking for duplicates on vendor, dept, 
# original value, and start date.

unique_counts = contracts.groupby(['procurement_id', 'vendor_name_norm', 'owner_org']).size()
id_org_uniques = unique_counts[unique_counts == 1]
id_org_amended = unique_counts[unique_counts > 1]

mult_id = (contracts
        .set_index(['procurement_id', 'vendor_name_norm', 'owner_org'])
        .loc[id_org_amended.index]
        .reset_index()
       )

# Of the ones that had only one record, check if there are multiples on value and date
val_date_uniques = contracts.set_index(['procurement_id', 'vendor_name_norm', 'owner_org']).loc[id_org_uniques.index].reset_index()
val_date_uniques = val_date_uniques.groupby(['vendor_name_norm', 'owner_org', 'original_value', 'contract_period_start']).size()
val_date_amended = val_date_uniques[val_date_uniques > 1]

mult_val_date = (contracts
        .set_index(['vendor_name_norm', 'owner_org', 'original_value', 'contract_period_start'])
        .loc[val_date_amended.index]
        .reset_index()
       )

# Redefine the unique ID for these values using buyer org, period start, vendor and original value
mult_val_date.proc_unique = (mult_val_date.owner_org + '_' 
                             + mult_val_date.contract_period_start.astype(str) + '_'
                             + mult_val_date.vendor_name_norm.str.split().str[0:2].str.join('_').str.lower() + '_'
                             + mult_val_date.original_value.astype(str).str[:4]
                            )

mults = pd.concat([mult_id, mult_val_date]).sort_values(['proc_unique', 'reference_number'])

In [11]:
mults.shape

(249044, 43)

In [14]:
# Get original contract value if that column is not null, else first contract value
first_orig_val = mults.groupby('proc_unique')[['original_value', 'contract_date']].first()
first_orig_val.columns = ['first_value', 'first_date']
first_orig_val.first_value = first_orig_val.first_value.fillna(mults.groupby('proc_unique').contract_value.first())

# Get last contract value and date
last_value = mults.groupby('proc_unique')[['contract_value', 'contract_date']].last()
last_value.columns = ['last_value', 'last_date']

num_amendments = mults.groupby('proc_unique').size().rename('num_amendments')
first_last = pd.concat([first_orig_val, last_value, num_amendments], axis=1)

# Calculate percentage change between first and last values
first_last['change'] = (first_last.last_value - first_last.first_value) / first_last.first_value * 100

In [15]:
# Join first record of each contract to the cost increase table and keep only records after 2012, removing anomalous contracts below $5000

amended = mults.groupby('proc_unique').first()[['procurement_id', 'vendor_name_norm', 'owner_org', 'owner_org_title', 'description_en',
                                                     'contract_date', 'fiscal_year', 'comments_en', 'commodity_type', 'commodity_code', 
                                                     'economic_object_code']]
amended = amended.join(first_last).query('contract_date.dt.year >= 2012 and first_value >= 5000')

In [16]:
amended.shape

(76236, 17)

In [41]:
# Save to parquet for faster loading

cost_changes.to_parquet('../data/amended_contracts.parquet')